In [4]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [5]:
model_path = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [10]:
dummy_model_input = tokenizer("This is a sample", return_tensors="pt")
print(dummy_model_input.keys())

dict_keys(['input_ids', 'attention_mask'])

In [11]:
model.eval()
# 변경할 onnx모델의 형식 지정
torch.onnx.export(
    model,							            # model 위치
    tuple(dummy_model_input.values()),			# input
    f="/root/onnx/model.onnx",		            # onnx모델 저장 경로
    input_names=['input_ids', 'attention_mask'],
    output_names=["logits"],
    dynamic_axes={
        "input_ids":{0: "batch_size", 1: "sequence"},
        "attention_mask":{0: "batch_size", 1: "sequence"},
        "logits":{0:'batch_size'},
                  },
    do_constant_folding=True, 
    opset_version=13,
)

/usr/local/lib/python3.8/dist-packages/transformers/models/distilbert/modeling_distilbert.py:223: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  mask, torch.tensor(torch.finfo(scores.dtype).min)


In [12]:
import onnxruntime as ort

In [34]:
import onnx 
import onnxruntime as ort 

onnx_path = "/root/onnx/model.onnx"
onnx_model = onnx.load(onnx_path)

sess = ort.InferenceSession(onnx_path)

# model_optimum = onnx.load("/root/onnx/optimum.onnx/model.onnx")
sess_optimum = ort.InferenceSession("/root/onnx/optimum.onnx/model.onnx")


In [36]:
onnx_input = {}
for input in sess.get_inputs():
    input_name = input.name
    input_data = dummy_model_input[input_name].numpy()
    onnx_input[input_name]=input_data

In [38]:
optimum_output = sess_optimum.run(None, onnx_input)

In [17]:

onnx_output = sess.run(None, onnx_input)

with torch.no_grad():
    model.to('cuda') 
    outputs = model(**dummy_model_input.to('cuda'))
    model.to('cpu')


SequenceClassifierOutput(loss=None, logits=tensor([[-2.5395,  2.6369]], device='cuda:0'), hidden_states=None, attentions=None)
[array([[-2.5394607,  2.6368706]], dtype=float32)]


In [28]:
import numpy as np 

np.allclose(outputs.logits.cpu(), onnx_output)

True

In [40]:
np.allclose(outputs.logits.cpu(), optimum_output)

True